In [1]:
import os

# Install Kaggle API
!pip install kaggle --upgrade --quiet  

# Set up Kaggle API Key
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

# Download dataset
!kaggle datasets download -d blastchar/telco-customer-churn

# Unzip dataset
import zipfile
with zipfile.ZipFile("telco-customer-churn.zip", 'r') as zip_ref:
    zip_ref.extractall("customer_churn_data")

print("Dataset downloaded and extracted successfully!")

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors

Dataset downloaded and extracted successfully!



  0%|          | 0.00/172k [00:00<?, ?B/s]
100%|##########| 172k/172k [00:01<00:00, 161kB/s]
100%|##########| 172k/172k [00:01<00:00, 161kB/s]


In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv("customer_churn_data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Convert 'totalcharges' to numeric (handling missing values)
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')

# Identify numeric columns only
numeric_cols = df.select_dtypes(include=['number']).columns

# Fill missing values only in numeric columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Drop duplicate rows if any
df.drop_duplicates(inplace=True)

# Display basic info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   seniorcitizen     7043 non-null   int64  
 3   partner           7043 non-null   object 
 4   dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   phoneservice      7043 non-null   object 
 7   multiplelines     7043 non-null   object 
 8   internetservice   7043 non-null   object 
 9   onlinesecurity    7043 non-null   object 
 10  onlinebackup      7043 non-null   object 
 11  deviceprotection  7043 non-null   object 
 12  techsupport       7043 non-null   object 
 13  streamingtv       7043 non-null   object 
 14  streamingmovies   7043 non-null   object 
 15  contract          7043 non-null   object 
 16  paperlessbilling  7043 non-null   object 


In [7]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect("customer_churn.db")
cursor = conn.cursor()

# Store data in SQL table
df.to_sql("customer_churn", conn, if_exists="replace", index=False)

print("Data successfully loaded into SQL Database!")

Data successfully loaded into SQL Database!


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Encode categorical variables
df['churn'] = df['churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Select relevant features
features = ['tenure', 'monthlycharges', 'totalcharges']
df = df[features + ['churn']].dropna()

# Split data
X = df[features]
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1036
           1       0.59      0.46      0.52       373

    accuracy                           0.77      1409
   macro avg       0.70      0.67      0.68      1409
weighted avg       0.76      0.77      0.76      1409

